In [48]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import prepState
from numpy import linalg as LA
from numpy import log as ln
from timeEvo import *
from qutip import *

# for using tex formatting and font in plots
#"""
plt.rcParams.update({"text.usetex": True,}) 
mpl.rcParams['text.latex.preamble'] = [r'\usepackage[utf8]{inputenc}\usepackage[T1]{fontenc}\usepackage{lmodern}\inputencoding{utf8}\usepackage{amsmath}\usepackage{amssymb}\usepackage{dsfont}\usepackage{mathtools}\usepackage{physics}']
mpl.rc('font', **{'family':'sans-serif', 'size':11})
cmap = mpl.cm.get_cmap('viridis')#['viridis', 'plasma', 'inferno', 'magma', 'cividis']

In [49]:
N=4
H_perf = generateStates(N,"perf")
hbar = ln(2)**2
n = 3200
dt = 1e-3
t = np.linspace(0, n*dt, n)

In [50]:
def thermal_state(beta):
    rho = (-beta * sigmaz()).expm()
    Z = rho.tr()
    rho = rho/Z
    return rho


def writetxt(filename, quantity):
    quantity = np.real(quantity)
    with open(str(filename) + ".txt", "w") as f:
        for element in quantity:
            f.write(str(element) + '\n')

In [51]:
## \dot{I}^2 max value as a function of alpha and position of max fidelity
def three_maxima(beta_one, beta_two, filename):
    alpha = 1.j / (4*np.cosh(beta_one)*np.cosh(beta_two))
    chi = alpha * tensor(Qobj([[0,1],[0,0]]), Qobj([[0,0],[1,0]])) - alpha * tensor(Qobj([[0,0],[1,0]]), Qobj([[0,1],[0,0]]))
    first_second_state = tensor(thermal_state(beta_one), thermal_state(beta_two))

    var_alpha_evo = []
    ratio = np.append(np.logspace(-5,-1,15, endpoint=False), np.linspace(.1, 1, 20, endpoint=True))

    ground_state = Qobj([[0,0],[0,1]])

    for i in ratio:
        corr_state = first_second_state + chi*i
        print(corr_state.eigenenergies(), alpha*i)
        print()
        var_alpha_system = tensor(corr_state, ground_state, ground_state, ground_state)
        time_evo = qutip.mesolve(H_perf, var_alpha_system, t, [], []).states
        var_alpha_evo.append(time_evo)

    ind = 0
    print("time evo done")
    print()

    filename = "maxima_with_diff_beta"+filename

    print(filename)

    with open("txtfiles/i_dot_sq_max/"+filename+".txt", 'a') as f:
        for alpha_evo in var_alpha_evo:
            kld_alpha = [entropy_relative(thermal_state(beta_one),alpha_evo[dt].ptrace(N)) for dt in range(len(t))]
            vn_alpha = [entropy_vn(alpha_evo[dt].ptrace(N), base=np.e) for dt in range(len(t))]

            i_dot_sq_alpha = np.square(np.diff(vn_alpha)/dt)
            bound = [-1.j *(np.pi/(3*hbar)) * (commutator(H_perf, alpha_evo[dt]).ptrace(N) * (sigmaz())).tr() for dt in range(1,len(t))]
            bound_diff = [np.real(bound[dt]) - i_dot_sq_alpha[dt] for dt in range(len(t)-1)]

            bound_diff_min = min(bound_diff)
            i_dot_sq_alpha_max = max(i_dot_sq_alpha)
            if beta_one > beta_two:
                kld_min_alpha = min(kld_alpha[:3000])
            else:
                kld_min_alpha = max(kld_alpha[:3138])
            kld_min_alpha_index=kld_alpha.index(kld_min_alpha)
            f.write(str(i_dot_sq_alpha_max)+'\t'+str(kld_min_alpha_index)+'\t'+str(bound_diff_min)+'\t'+str(ratio[ind])+'\t'+str(beta_one)+'\t'+str(beta_two)+'\n')
            print(str(i_dot_sq_alpha_max)+'\t'+str(kld_min_alpha_index)+'\t'+str(bound_diff_min)+'\t'+str(ratio[ind])+'\t'+str(beta_one)+'\t'+str(beta_two))
            ind += 1


In [ ]:
betas = np.insert(np.logspace(-.3,ln(2)/ln(10),4, endpoint=False),0,1/300)
beta_1 = np.logspace(-.3,ln(2)/ln(10),4, endpoint=False)
beta_2 = np.insert(np.logspace(-.3,ln(2)/ln(10),4, endpoint=False),0,1/300)

#beta_1, beta_2 = np.meshgrid(beta_1, beta_2)

#i, j = 1,0
#for beta_one in beta_1:
#    for beta_two in beta_2:
#        three_maxima(beta_one, beta_two, str(i)+str(j))
#        j+=1
#    i+=1
#    j=0

In [52]:
ij = []

i,j = 0.5011872336272722, 0.0033333333333333335
three_maxima(i, i, str(1)+str(1))
betas = np.insert(np.logspace(-.3,ln(2)/ln(10), 3, endpoint=False), [0,0], [1/300,.25])
#print(ij, len(ij), [expect(thermal_state(beta), sigmaz())for beta in betas])

#for i,j in ij:
    #three_maxima(betas[i], betas[j], str(i)+str(j))

[0.13378996 0.13468487 0.36454338 0.36698179] 2.21581764267257e-06j

[0.13378996 0.13468487 0.36454338 0.36698179] 4.094498182137246e-06j

[0.13378996 0.13468487 0.36454338 0.36698179] 7.566017636408247e-06j

[0.13378996 0.13468487 0.36454338 0.36698179] 1.3980864156730457e-05j

[0.13378996 0.13468487 0.36454338 0.36698179] 2.5834536999802942e-05j

[0.13378996 0.13468486 0.36454339 0.36698179] 4.773834396158454e-05j

[0.13378996 0.13468484 0.36454342 0.36698179] 8.821328921868972e-05j

[0.13378996 0.13468476 0.3645435  0.36698179] 0.00016300490861270974j

[0.13378996 0.13468448 0.36454378 0.36698179] 0.00030120858735883466j

[0.13378996 0.13468352 0.36454473 0.36698179] 0.000556588227132877j

[0.13378996 0.13468027 0.36454798 0.36698179] 0.0010284914427551187j

[0.13378996 0.13466916 0.36455909 0.36698179] 0.0019004977041456054j

[0.13378996 0.13463123 0.36459702 0.36698179] 0.0035118342976069895j

[0.13378996 0.13450181 0.36472644 0.36698179] 0.006489342295413728j

[0.13378996 0.13406